<a href="https://colab.research.google.com/github/saylendrayasin/Practice_Making_Sentiment_Analysis/blob/main/DataCrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Crawling data tanpa API
!pip install -q snscrape==0.3.4

In [ ]:
# Import library
import datetime as dt
import re
import string
import snscrape.modules.twitter as sntwitter
import tweepy
import pandas as pd
import ast
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from multiprocessing import Pool
from functools import partial
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Mengambil Data**

In [ ]:
#Menentukan max Tweets
maxTweets = 5000
tweets_lists2 = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper('Senin since:2022-01-01 until:2022-06-20 lang:id').get_items()):
  if i>maxTweets:
    break
  tweets_lists2.append([tweet.date, tweet.content, tweet.username])

In [ ]:
tweets_df2 = pd.DataFrame(tweets_lists2, columns=['Datetime', 'Text', 'Username'])

tweets_df2.head()

# **Simpan file ke dalam folder**

In [ ]:
tweets_df2.to_csv('drive/My Drive/Data Set/crawlingSenin5000.csv', sep=',', index=False)

NameError: ignored

In [ ]:
tweets_data = pd.read_csv('drive/My Drive/Data Set/crawlingSenin5000.csv')
tweets = tweets_data[['Datetime', 'Username', 'Text']]
tweets.head()

,Datetime,Username,Text
0,2022-06-19 23:59:52+00:00,vaneshaasya,Selamat hari senin. Tetap semangat menjalani h...
1,2022-06-19 23:59:52+00:00,KubuPolsek,"Pelayanan Keamanan, Bhabinkamtibmas Desa Tiany..."
2,2022-06-19 23:59:51+00:00,Yosandewi,Selamat hari senin 🫶
3,2022-06-19 23:59:49+00:00,reysd0minstion,selamat hari senin ckep
4,2022-06-19 23:59:44+00:00,marchwrj,selamat hari senin semoga harimu selalu bigwin


# **Slang**

In [ ]:
!wget https://github.com/evanmartua34/Twitter-COVID19-Indonesia-Sentiment-Analysis---Lexicon-Based/raw/master/cleaning_source/update_combined_slang_words.txt
!ls

--2022-06-22 03:22:51--  https://github.com/evanmartua34/Twitter-COVID19-Indonesia-Sentiment-Analysis---Lexicon-Based/raw/master/cleaning_source/update_combined_slang_words.txt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/evanmartua34/Twitter-COVID19-Indonesia-Sentiment-Analysis---Lexicon-Based/master/cleaning_source/update_combined_slang_words.txt [following]
--2022-06-22 03:22:51--  https://raw.githubusercontent.com/evanmartua34/Twitter-COVID19-Indonesia-Sentiment-Analysis---Lexicon-Based/master/cleaning_source/update_combined_slang_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [ ]:
import ast
slangJson = {}

with open('update_combined_slang_words.txt', 'r') as slangFile:
  slangJson = ast.literal_eval(slangFile.read())

print(slangJson)

{'@': 'di', 'abis': 'habis', 'ad': 'ada', 'adlh': 'adalah', 'afaik': 'as far as i know', 'ahaha': 'haha', 'aj': 'saja', 'ajep-ajep': 'dunia gemerlap', 'ak': 'saya', 'akika': 'aku', 'akkoh': 'aku', 'akuwh': 'aku', 'alay': 'norak', 'alow': 'halo', 'ambilin': 'ambilkan', 'ancur': 'hancur', 'anjrit': 'anjing', 'anter': 'antar', 'ap2': 'apa-apa', 'apasih': 'apa sih', 'apes': 'sial', 'aps': 'apa', 'aq': 'saya', 'aquwh': 'aku', 'asbun': 'asal bunyi', 'aseekk': 'asyik', 'asekk': 'asyik', 'asem': 'asam', 'aspal': 'asli tetapi palsu', 'astul': 'asal tulis', 'ato': 'atau', 'au ah': 'tidak mau tahu', 'awak': 'saya', 'ay': 'sayang', 'ayank': 'sayang', 'b4': 'sebelum', 'bakalan': 'akan', 'bandes': 'bantuan desa', 'bangedh': 'banget', 'banpol': 'bantuan polisi', 'banpur': 'bantuan tempur', 'basbang': 'basi', 'bcanda': 'bercanda', 'bdg': 'bandung', 'begajulan': 'nakal', 'beliin': 'belikan', 'bencong': 'banci', 'bentar': 'sebentar', 'ber3': 'bertiga', 'beresin': 'membereskan', 'bete': 'bosan', 'beud': 

In [ ]:
cth = "Bagaimana jayus \x86 :) untuk melakukan dlu #halhal sesuatu yang tidak @dapat diprediksi yang   membuat \nkalian \n\nbingung dan pusing 1222 www.testing.com"
cth = word_tokenize(cth)

for i in range(len(cth)):
  if cth[i] in slangJson:
    cth[i] = slangJson[cth[i]]

print(cth)

['Bagaimana', 'tidak lucu', '\x86', ':', ')', 'untuk', 'melakukan', 'dulu', '#', 'halhal', 'sesuatu', 'yang', 'tidak', 'di', 'dapat', 'diprediksi', 'yang', 'membuat', 'kalian', 'bingung', 'dan', 'pusing', '1222', 'www.testing.com']


In [ ]:
gkounter = 0

def cleaningText(text):
  global gkounter
  text = re.sub(r'@[A-Za-z0-9]+', '',str(text)) # hapus mention
  text = re.sub(r'#[A-Za-z0-9]+', '',str(text)) # hapus hastag
  text = re.sub(r'RT[\s]', '',str(text)) # hapus RT
  text = re.sub(r"(http\S+)|(www\.[^\s]+)", '',str(text)) # hapus link
  text = re.sub(r'[0-9]+', '',str(text)) # hapus angka
  text = re.sub(r'[^A-Za-z0-9 \t]|(\w+:\/\/\S+)+','', str (text)) # hapus emoticon
  text = text.replace('\n', ' ') # ganti baris baru dengan spasi
  text = re.sub(r'(\\u[0-9A-Fa-f]+)', r'',str(text)) # hapus unicode seperti \uu02c
  text = re.sub(r'[^\x00-\x7f]', r'',str(text)) # hapus unicode x96
  text = re.sub('@[^\s]+','',str(text)) # hapus user yang dimention
  text = re.sub('[\s]+', ' ',str(text)) # hapus space lebih dari 1 kali
  text = re.sub('[\n]+', ' ',str(text)) # hapus enter lebih dari 1 jadi spasi
  text = text.translate(str.maketrans('', '', string.punctuation)) # hapus semua tanda baca
  text = text.strip(' ') # hapus spasi kosong yang ada di awal dan di akhir teks
  print(f"cleaned {gkounter} text")
  gkounter += 1
  return text

In [ ]:
# fungsi untuk merubah semua ke huruf kecil
def casefoldingText(text):
  global gkounter
  text = text.lower()
  print (f"casefolded {gkounter} text")
  gkounter += 1
  return text

In [ ]:
# fungsi Tokenizing
def tokenizingText(text):
  global gkounter
  text = word_tokenize(text)
  print (f"tokenized {gkounter} text")
  gkounter += 1
  return text

Perbedaan split dan tokenize adalah 

split akan membagi kata berdasarkan kata karakter yang mengikuti kata tersebut

tokenize akan membagi kata berdsasarkan token jadi jika ada karakter diblakang kata maka akan dihitung sendiri

In [ ]:
def normalizeSlangWords(cth):  #normalisasi kata alay/slang menjadi kata baku bahasa indonesia
    global gkounter
    for i in range(len(cth)):
      if cth[i] in slangJson:
        cth[i] = slangJson[cth[i]]
    print(f"normalized slang words {gkounter} text")
    gkounter += 1
    return cth

In [ ]:
def filteringText(text):
  global gkounter
  listStopwords = set(stopwords.words('indonesian'))
  filtered = []
  for txt in text:
    if txt not in listStopwords:
      filtered.append(txt)
  text = filtered
  print(f"filtered {gkounter} text")
  gkounter += 1
  return text

Perbedaan list dan set

dalam python set lebih cepat dibandingkan dengan list

In [ ]:
def stemmingText(text):
  global gkounter
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  text = [stemmer.stem(word) for word in text]
  print(f"stemmed {gkounter} text")
  gkounter += 1
  return text

In [ ]:
def toSentence(list_words):
  sentence = ' '.join(word for word in list_words)
  return sentence

In [ ]:
tweet_cth = "Bagaimana jayus \x86 :) untuk melakukan dlu #halhal sesuatu yang tidak @dapat diprediksi yang   membuat \nkalian \n\nbingung dan pusing 1222 www.testing.com"
gkounter = 0
tweet_cth = cleaningText(tweet_cth)
# tweet_cth = tweet_cth.encode('ascii', errors='ignore')
tweet_cth

In [ ]:
gkounter = 0
tweet_stem = stemmingText(tweet_lwr)
tweet_stem

In [ ]:
gkounter = 0
tweet_lwr = casefoldingText(tweet_cth)
tweet_lwr

In [ ]:
gkounter = 0
tweet_tkn = tokenizingText(tweet_lwr)
tweet_tkn

In [ ]:
tweets
gkounter = 0
tweets['Clean_Text'] = tweets['Text'].apply(cleaningText)
gkounter = 0
tweets['Clean_Text'] = tweets['Clean_Text'].apply(casefoldingText)
gkounter = 0
tweets['Preprocess_Text'] = tweets['Clean_Text'].apply(tokenizingText)
gkounter = 0
tweets['Preprocess_Text'] = tweets['Preprocess_Text'].apply(normalizeSlangWords)
gkounter = 0
tweets['Preprocess_Text'] = tweets['Preprocess_Text'].apply(filteringText)



Output streaming akan dipotong hingga 5000 baris terakhir.
filtered 1 text
filtered 2 text
filtered 3 text
filtered 4 text
filtered 5 text
filtered 6 text
filtered 7 text
filtered 8 text
filtered 9 text
filtered 10 text
filtered 11 text
filtered 12 text
filtered 13 text
filtered 14 text
filtered 15 text
filtered 16 text
filtered 17 text
filtered 18 text
filtered 19 text
filtered 20 text
filtered 21 text
filtered 22 text
filtered 23 text
filtered 24 text
filtered 25 text
filtered 26 text
filtered 27 text
filtered 28 text
filtered 29 text
filtered 30 text
filtered 31 text
filtered 32 text
filtered 33 text
filtered 34 text
filtered 35 text
filtered 36 text
filtered 37 text
filtered 38 text
filtered 39 text
filtered 40 text
filtered 41 text
filtered 42 text
filtered 43 text
filtered 44 text
filtered 45 text
filtered 46 text
filtered 47 text
filtered 48 text
filtered 49 text
filtered 50 text
filtered 51 text
filtered 52 text
filtered 53 text
filtered 54 text
filtered 55 text
filtered 56 tex

In [ ]:
tweets

,Datetime,Username,Text,Clean_Text,Preprocess_Text
0,2022-06-19 23:59:52+00:00,vaneshaasya,Selamat hari senin. Tetap semangat menjalani h...,selamat hari senin tetap semangat menjalani hi...,"[selamat, senin, semangat, menjalani, hidup, p..."
1,2022-06-19 23:59:52+00:00,KubuPolsek,"Pelayanan Keamanan, Bhabinkamtibmas Desa Tiany...",pelayanan keamanan bhabinkamtibmas desa tianya...,"[pelayanan, keamanan, bhabinkamtibmas, desa, t..."
2,2022-06-19 23:59:51+00:00,Yosandewi,Selamat hari senin 🫶,selamat hari senin,"[selamat, senin]"
3,2022-06-19 23:59:49+00:00,reysd0minstion,selamat hari senin ckep,selamat hari senin ckep,"[selamat, senin, ckep]"
4,2022-06-19 23:59:44+00:00,marchwrj,selamat hari senin semoga harimu selalu bigwin,selamat hari senin semoga harimu selalu bigwin,"[selamat, senin, semoga, harimu, bigwin]"
...,...,...,...,...,...
4996,2022-06-19 15:28:09+00:00,Androngehe,@hrdbacot Kalau kamu menjelang hari senin nger...,kalau kamu menjelang hari senin ngerasain cema...,"[menjelang, senin, ngerasain, cemas, overthink..."
4997,2022-06-19 15:28:04+00:00,revaldstyn,"@Celine_JKT48 thanks, apa itu senin? mahasiswa...",jkt thanks apa itu senin mahasiswa akhir semua...,"[jakarta, terima kasih, senin, mahasiswa, dian..."
4998,2022-06-19 15:28:04+00:00,verpoleon,"@Benthve ㅤ\n“Nah, setelah pengumuman tadi, akh...",nah setelah pengumuman tadi akhirnya kita samp...,"[pengumuman, ujung, acara, malam, dilanjut, be..."
4999,2022-06-19 15:28:01+00:00,jihanc__,belum ngantuk tapi besok senin😩,belum ngantuk tapi besok senin,"[ngantuk, besok, senin]"


In [ ]:
!apt install screenfetch
!screenfetch

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  bc giblib1 libid3tag0 libimlib2 scrot
The following NEW packages will be installed:
  bc giblib1 libid3tag0 libimlib2 screenfetch scrot
0 upgraded, 6 newly installed, 0 to remove and 49 not upgraded.
Need to get 375 kB of archives.
After this operation, 1,280 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 bc amd64 1.07.1-2 [86.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libid3tag0 amd64 0.15.1b-13 [31.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libimlib2 amd64 1.4.10-1 [173 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 giblib1 amd64 1.2.4-11 [17.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu 

In [ ]:
def parallelize(data, func, num_of_processes=2):
  data_split = np.array_split(data, num_of_processes)

  pool = Pool(num_of_processes)

  data = pd.concat(pool.map(func, data_split))

  pool.close()

  pool.join()
  
  return data


def run_on_subset(func, data_subset):
  return data_subset.apply(func)

def parallelize_on_rows(data, func, num_of_processes=2):
  return parallelize(data, partial(run_on_subset, func), num_of_processes)

tweets.drop_duplicates(subset = 'Clean_Text', inplace = True)
print(tweets.count)

gkounter = 0

tweets['Preprocess_Text'] = parallelize_on_rows(tweets['Preprocess_Text'], stemmingText)

print("exporting to csv")

tweets.to_csv('drive/My Drive/Data Set/crawlingSenin5000-cleaned.csv', index=False, header=True, index_label=None)

tweets

<bound method DataFrame.count of                        Datetime        Username  \
0     2022-06-19 23:59:52+00:00     vaneshaasya   
1     2022-06-19 23:59:52+00:00      KubuPolsek   
2     2022-06-19 23:59:51+00:00       Yosandewi   
3     2022-06-19 23:59:49+00:00  reysd0minstion   
4     2022-06-19 23:59:44+00:00        marchwrj   
...                         ...             ...   
4996  2022-06-19 15:28:09+00:00      Androngehe   
4997  2022-06-19 15:28:04+00:00      revaldstyn   
4998  2022-06-19 15:28:04+00:00       verpoleon   
4999  2022-06-19 15:28:01+00:00        jihanc__   
5000  2022-06-19 15:28:00+00:00    dhernilarosa   

                                                   Text  \
0     Selamat hari senin. Tetap semangat menjalani h...   
1     Pelayanan Keamanan, Bhabinkamtibmas Desa Tiany...   
2                                  Selamat hari senin 🫶   
3                               selamat hari senin ckep   
4        selamat hari senin semoga harimu selalu bigwin   


,Datetime,Username,Text,Clean_Text,Preprocess_Text
0,2022-06-19 23:59:52+00:00,vaneshaasya,Selamat hari senin. Tetap semangat menjalani h...,selamat hari senin tetap semangat menjalani hi...,"[selamat, senin, semangat, jalan, hidup, penuh..."
1,2022-06-19 23:59:52+00:00,KubuPolsek,"Pelayanan Keamanan, Bhabinkamtibmas Desa Tiany...",pelayanan keamanan bhabinkamtibmas desa tianya...,"[layan, aman, bhabinkamtibmas, desa, tianyar, ..."
2,2022-06-19 23:59:51+00:00,Yosandewi,Selamat hari senin 🫶,selamat hari senin,"[selamat, senin]"
3,2022-06-19 23:59:49+00:00,reysd0minstion,selamat hari senin ckep,selamat hari senin ckep,"[selamat, senin, ckep]"
4,2022-06-19 23:59:44+00:00,marchwrj,selamat hari senin semoga harimu selalu bigwin,selamat hari senin semoga harimu selalu bigwin,"[selamat, senin, moga, hari, bigwin]"
...,...,...,...,...,...
4996,2022-06-19 15:28:09+00:00,Androngehe,@hrdbacot Kalau kamu menjelang hari senin nger...,kalau kamu menjelang hari senin ngerasain cema...,"[jelang, senin, ngerasain, cemas, overthinking..."
4997,2022-06-19 15:28:04+00:00,revaldstyn,"@Celine_JKT48 thanks, apa itu senin? mahasiswa...",jkt thanks apa itu senin mahasiswa akhir semua...,"[jakarta, terima kasih, senin, mahasiswa, angg..."
4998,2022-06-19 15:28:04+00:00,verpoleon,"@Benthve ㅤ\n“Nah, setelah pengumuman tadi, akh...",nah setelah pengumuman tadi akhirnya kita samp...,"[umum, ujung, acara, malam, lanjut, besok, pag..."
4999,2022-06-19 15:28:01+00:00,jihanc__,belum ngantuk tapi besok senin😩,belum ngantuk tapi besok senin,"[ngantuk, besok, senin]"


In [ ]:
# tweets[:100] #Mengambil 100 data dari atas
# tweets[-100:] #Mengambil 100 data dari bawah || 
# Bedanya dengan head dan tail adalah pada head dan tail hanya melihat data sedangkan list mengambil data